In [6]:
import matplotlib.pyplot as plt   
import numpy as np   
import pandas as pd 
from sklearn.preprocessing import MinMaxScaler
from tensorflow import keras

from keras.layers.core import Dense,Activation,Dropout
from keras.layers.recurrent import LSTM 
from keras.models import Sequential

ModuleNotFoundError: No module named 'keras.layers.recurrent'

In [ ]:
data = pd.read_csv('Data/stock-data-2000-2017.csv')
data = data[['Open','High','Low','Volume','Close']]
data.head()

In [ ]:
sequence_length = 21
n_features = len(data.columns)
val_ratio = 0.1
n_epochs = 300
batch_size = 512

In [ ]:
data = data.as_matrix()
data_processed=[]
for index in range(len(data)-sequence_length):
    data_processed.append(data[index:index+sequence_length])
data_processed = np.array(data_processed)

In [ ]:
val_split  = round((1-val_ratio)*data_processed.shape[0])
train = data_processed[:int(val_split),:]
val = data_processed[int(val_split):,:]
print('Training data:{}'.format(train.shape))
print('vali data:{}'.format(val.shape))

In [ ]:
train_samples, train_nx, train_ny = train.shape
val_samples, val_nx, val_ny = val.shape

train = train.reshape((train_samples, train_nx * train_ny))
val = val.reshape((val_samples, val_nx * val_ny))

preprocessor = MinMaxScaler().fit(train)
train = preprocessor.transform(train)
val = preprocessor.transform(val)

train = train.reshape((train_samples, train_nx, train_ny))
val = val.reshape((val_samples, val_nx, val_ny))

In [ ]:
X_train = train[:, : -1]
y_train = train[:, -1][: ,-1]
X_val = val[:, : -1]
y_val = val[:, -1][ : ,-1]

X_train = np.reshape(X_train, (X_train.shape[0], X_train.shape[1], n_features))
X_val = np.reshape(X_val, (X_val.shape[0], X_val.shape[1], n_features))

In [ ]:
model = Sequential([
    LSTM(input_shape=(X_train.shape[1:]),units = 128,return_sequences=True),
    Dropout(0.5),
    LSTEM(128,return_sequences=False),
    Dropout(0.25),
    Dense(units=1),
    Activation('linear')
    
])
model.compile(loss="mse",optimizer("adam"))
history = model.fit(
    X_train,
    y_train,
    batch_size = batch_size,
    epochs=n_epochs,
    verbose=2
)

In [ ]:
preds_val = model.predict(X_val)
diff = []
for i in range(len(y_val)):
    pred = preds_val[i][0]
    diff.append(y_val[i] - pred)

In [ ]:
real_min = preprocessor.data_min_[104]
real_max = preprocessor.data_max_[104]
print(preprocessor.data_min_[104])
print(preprocessor.data_max_[104])

preds_real = preds_val * (real_max - real_min) + real_min
y_val_real = y_val * (real_max - real_min) + real_min

In [ ]:
plt.plot(preds_real, label='Predictions')
plt.plot(y_val_real, label='Actual values')
plt.xlabel('test')
plt.legend(loc=0)
plt.show()